# Bus Admittance Matrix

In [1]:
import numpy as np

import andes

from andes.thirdparty.npfunc import safe_div

from andes.linsolvers.scipy import spmatrix_to_csc

import scipy.sparse as sps

In [2]:
ss = andes.load(andes.get_case('kundur/kundur.raw'),
                setup=True, no_output=True, default_config=True)

Generating code for 1 models on 12 processes.


ANDES allows bus admittance matrix calculation.

In [3]:
Y = spmatrix_to_csc(ss.Line.build_y()).todense()
Y.shape

(10, 10)

In [4]:
ss.PFlow.run()

True

In [5]:
stg = ss.StaticGen.find_idx(
    keys='bus', values=ss.Bus.idx.v, allow_all=True, default=None, allow_none=True)
p_gen = ss.StaticGen.get(src='a', attr='e', idx=stg,
                         allow_none=True, default=0)
q_gen = ss.StaticGen.get(src='v', attr='e', idx=stg,
                         allow_none=True, default=0)

r_gen = ss.StaticGen.get(
    src='ra', attr='v', idx=stg, allow_none=True, default=0)
x_gen = ss.StaticGen.get(
    src='xs', attr='v', idx=stg, allow_none=True, default=0)

I_gen = safe_div(p_gen + 1j * q_gen, r_gen + 1j * x_gen)
I_gen

array([-4.37853526+29.07211691j, -9.12192109+28.j        ,
       -9.29537908+28.j        , -4.24364233+28.j        ,
        0.         +0.j        ,  0.         +0.j        ,
        0.         +0.j        ,  0.         +0.j        ,
        0.         +0.j        ,  0.         +0.j        ])

In [6]:
V1 = ss.Bus.get(src='v', attr='v', idx=ss.Line.bus1.v) * np.exp(1j *
                                                                ss.Bus.get(src='a', attr='v', idx=ss.Line.bus1.v) * np.pi / 180)
V2 = ss.Bus.get(src='v', attr='v', idx=ss.Line.bus2.v) * np.exp(1j *
                                                                ss.Bus.get(src='a', attr='v', idx=ss.Line.bus2.v) * np.pi / 180)
r = ss.Line.r.v
x = ss.Line.x.v
# Calculate the current flowing through the line
I_line_through = safe_div(V1 - V2, r + 1j * x)

# --- Cft ---
nb = ss.Bus.n
nl = ss.Line.n

# line indices: idx -> uid
idx_line = ss.Line.idx.v
u_line = ss.Line.get(src='u', attr='v', idx=idx_line)
on_line = np.flatnonzero(u_line)
on_line_idx = [idx_line[i] for i in on_line]
on_line_bus1 = ss.Line.get(src='bus1', attr='v', idx=on_line_idx)
on_line_bus2 = ss.Line.get(src='bus2', attr='v', idx=on_line_idx)

data_line = np.ones(2*len(on_line_idx))
data_line[len(on_line_idx):] = -1
row_line = np.array([ss.Bus.idx2uid(x) for x in on_line_bus1 + on_line_bus2])
col_line = np.array([ss.Line.idx2uid(x) for x in on_line_idx + on_line_idx])
Cft = sps.csr_matrix((data_line, (row_line, col_line)), (nb, nl)).toarray()

I_line = Cft @ I_line_through
I_line

array([ 0.25279928-1.36315255j,  0.34819046-2.54640974j,
        0.34377786-2.59071275j,  0.24065267-1.33158988j,
       -0.06488381+0.8113498j , -0.15261117+1.85117031j,
       -0.34003937+1.22118412j, -0.4354037 +1.44239613j,
       -0.13695573+1.76345288j, -0.05552648+0.74231166j])

In [7]:
I_bus = I_line + I_gen

In [8]:
I_bus

array([-4.12573598+27.70896436j, -8.77373063+25.45359026j,
       -8.95160122+25.40928725j, -4.00298966+26.66841012j,
       -0.06488381 +0.8113498j , -0.15261117 +1.85117031j,
       -0.34003937 +1.22118412j, -0.4354037  +1.44239613j,
       -0.13695573 +1.76345288j, -0.05552648 +0.74231166j])